# Experimentovanie a vyhodnocovanie

Miroslav Čulík a Andrej Gáfrik

Otázky:
- MSE vs R2 - ak odstránime z input df inštancie s cenou viac ako threshold (815000), vieme znižiť MSE, RMSE ale spadne nam aj R2 - ktora metrika by mala byť prioritná
- primitívny prediktor - pred všetky inštancie budeme predikovať priemernú cenu a porovnáme MSE, RMSE, R2,...
- môžu sa odstraňovať outlieri po transformácii? nemali by sme nahrádzať hodnoty outlierov takými, akými sme ich identifikovali?
- RandomSearch pri RF: Ktorý model je lepší - ten čo má nižšiu delta_R2 alebo ten čo má najvyšší R2_test?

TODOs:
- prekopčiť veci z minulých správ do finálnej správy na overleafe https://www.overleaf.com/project/5e7101e344e1ac0001e3041b
- Experimenty
    - DM metóda: experimentovanie, nastavenie parametrov, trénovanie, testovanie, ...
    - vyhodnotenie: vyhodnotenie výsledkov, porovnanie metód (aj s publikovanými prácami)
- Zhrnutie
    - zhodnotenie toho, čo sa v rámci projektu podarilo
    - náčrt ďalšieho vhodného smerovania projektu
- Literatúra: zoznam použitej literatúry a ich citovanie v správe


- ~~pridat nove modely~~
    - ~~RF~~
    - ~~SVR~~

- ~~hyperparameter tuning~~
    - ~~manual pri RF~~
    - ~~gridsearch~~
    - ~~random search na RF a podla neho gridsearch~~
- ~~primitívny prediktor~~
- ~~refactoring~~
- feature importance(RF, Decision Tree)
- upravit pipeline pre data mensie ako threshold, ktory je r_whisker
- optional: skusit ako to bude ked dame namiesto boxcox pri cene log transformaciu


In [1]:
cd ..

C:\Users\Mirec\Desktop\08_fiit_ls_2019_2020\OZNAL\projekt\oznal_project


In [2]:
import seaborn as sns
import random
import itertools
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.special import inv_boxcox
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from numpy.polynomial.polynomial import polyfit

from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDRegressor

from src import analysis, preprocessing2, feature_selection2, metrics2, tuning3

pd.set_option('max_rows', None)
pd.set_option('max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
path_housing = "data\\kc_house_data.csv"
df = pd.read_csv(path_housing, sep=",")
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.00000,3,1.00000,1180,5650,1.00000,0,0,3,7,1180,0,1955,0,98178,47.51120,-122.25700,1340,5650
1,6414100192,20141209T000000,538000.00000,3,2.25000,2570,7242,2.00000,0,0,3,7,2170,400,1951,1991,98125,47.72100,-122.31900,1690,7639
2,5631500400,20150225T000000,180000.00000,2,1.00000,770,10000,1.00000,0,0,3,6,770,0,1933,0,98028,47.73790,-122.23300,2720,8062
3,2487200875,20141209T000000,604000.00000,4,3.00000,1960,5000,1.00000,0,0,5,7,1050,910,1965,0,98136,47.52080,-122.39300,1360,5000
4,1954400510,20150218T000000,510000.00000,3,2.00000,1680,8080,1.00000,0,0,3,8,1680,0,1987,0,98074,47.61680,-122.04500,1800,7503


In [4]:
_,threshold = analysis.get_whiskers(df["price"])
# threshold = 815000
new_df = df[df["price"]<=threshold]
print(len(new_df))

20467


In [5]:
X_train, X_valid, X_test, y_train, y_valid, y_test, price_lambda = preprocessing2.run_pipeline(new_df)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [6]:
X_train.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode_1,zipcode_2,zipcode_3,zipcode_4,zipcode_5,zipcode_6,zipcode_7,lat,long,sqft_living15,sqft_lot15,recon_age,price_per_sqft
3714,6127600110,4,2.00000,16.56271,4.25500,1.50000,0,0,3,7,4.10811,0.00000,1945,0,0,0,0,0,0,0,1,47.67800,-122.26900,7.55486,4.06086,69,0.02223
12496,1982201345,2,1.00000,13.97492,4.20736,1.00000,0,0,4,7,3.92192,0.00000,1944,0,0,0,0,0,0,1,0,47.66390,-122.36400,7.04752,4.00023,70,0.02981
8326,2493200195,3,1.75000,18.53728,4.28525,1.00000,0,0,4,8,4.07734,31.62278,1948,0,0,0,0,0,0,1,1,47.52740,-122.38400,7.52833,4.04062,66,0.02424
6711,5652600185,3,1.75000,18.29393,4.35126,2.00000,0,0,5,8,4.11345,26.26785,1923,0,0,0,0,0,0,0,1,47.69540,-122.29200,7.35883,4.14516,91,0.02223
9596,472000620,3,2.50000,18.99565,4.20321,1.00000,0,0,4,9,4.13846,30.00000,1951,0,0,0,0,0,1,0,0,47.68330,-122.40000,7.77486,4.01524,63,0.02506


## Experimenty

Po predspracovaní dát nami vytvorenou pipeline, sme mali dáta rozdelené v pomere 70:20:10 (trénovacia : validačná : testovacia množina). Použili sme implementácie algoritmov z modulu sklearn. Najlepšie výsledky podľa metriky R$^2$ dosiahol model polynomiálnej regresie 3. stupňa s použitím selekcie čŕt RFE (0.916 na trénovacej a 0.828 na testovacej množine), druhý najlepší výsledok podľa tejto metriky dosiahol model regresného rozhodovacieho stromu(0.861 na trénovacej a 0.802 na testovacej množine). V prípade modelu jednoduchej lineárnej regresie sme vyskúšali aj 5, resp. 10 - násobnú krížovú validáciu, na porovnanie výsledkov s modelom jednoduchej lineárnej regresie trénovaného tradičným spôsobom, avšak tento prístup nám nepreukázal žiaden signifikantný rozdiel vo výsledkoch pri modeloch jednoduchej lineárnej regresie.
 

### Primitívny prediktor
Náš primitívny prediktor bude slúžiť na porovnanie s našimi modelmi pri metrikách MSE, RMSE a RMSLE. Predikovať budeme pre každú inštanciu priemernú cenu podľa trénovacej množiny.

In [7]:
mean = np.mean(y_train)
y_hat = pd.Series([mean for _ in range(len(y_valid))])

mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))

print('Mean Squared Error', f'{round(mean_squared_error, 5):,}')
print('Root Mean Squared Error', f'{round(np.sqrt(mean_squared_error),5):,}')
print('Root Mean Squared Log Error', f'{round(np.sqrt(rmlse),5):,}')

Mean Squared Error 44,308,751,649.17746
Root Mean Squared Error 210,496.44094
Root Mean Squared Log Error 0.44688


In [8]:
np.mean(inv_boxcox(y_train, price_lambda))

476437.60269165935

### Jednoduchá lineárna regresia - sqft_living

In [26]:
reg=linear_model.LinearRegression()
x_train=np.array(X_train['sqft_living']).reshape(-1,1)
y_train=np.array(y_train).reshape(-1,1)
reg.fit(x_train,y_train)

x_test=np.array(X_valid['sqft_living']).reshape(-1,1)
y_test=np.array(y_valid).reshape(-1,1)
pred=reg.predict(x_test)
print('linear model - sqft_living')

metrics2.evaluate(reg, x_train, y_train, x_test, y_test, pred, price_lambda)

linear model - sqft_living
Mean Squared Error 29,287,497,750.05915
Root Mean Squared Error 171,135.90433
Root Mean Squared Log Error 0.3658
R squared training 0.39
R squared testing 0.331
Adjusted-R squared training 0.39
Adjusted-R squared testing 0.331
intercept [21.96834431]
coefficient [[1.30406286]]


#### k-násobná krížová validácia pri jednoduchej lineárnej regresii

In [7]:
reg=linear_model.LinearRegression()
k = 5

X_try = pd.concat([X_train['sqft_living'], X_valid['sqft_living']])
y_try = np.concatenate((np.array(y_train).reshape(-1,1), np.array(y_valid).reshape(-1,1)), axis=0)

scores = cross_val_score(reg,  np.array(X_try).reshape(-1,1), np.array(y_try).reshape(-1,1), cv=k)
print("Cross-validated scores:", scores.mean())
k = 10
scores = cross_val_score(reg,  np.array(X_try).reshape(-1,1), np.array(y_try).reshape(-1,1), cv=k)
print("Cross-validated scores:", scores.mean())

Cross-validated scores: 0.4388515347655395
Cross-validated scores: 0.4385100197794961


### Jednoduchá lineárna regresia - grade

In [27]:
reg=linear_model.LinearRegression()
x_train=np.array(X_train['grade']).reshape(-1,1)
y_train=np.array(y_train).reshape(-1,1)
reg.fit(x_train,y_train)

x_test=np.array(X_valid['grade']).reshape(-1,1)
y_test=np.array(y_valid).reshape(-1,1)
pred=reg.predict(x_test)
print('linear model - grade')

metrics2.evaluate(reg, x_train, y_train, x_test, y_test, pred, price_lambda)

linear model - grade
Mean Squared Error 28,690,872,369.15526
Root Mean Squared Error 169,383.80197
Root Mean Squared Log Error 0.36174
R squared training 0.41
R squared testing 0.346
Adjusted-R squared training 0.41
Adjusted-R squared testing 0.346
intercept [27.78175155]
coefficient [[2.24798499]]


### Viacnásobná lineárna regresia - part1

In [28]:
features1 = feature_selection2.feature_filter(X_train, y_train, threshold=0.5)
reg=linear_model.LinearRegression()
reg.fit(X_train[features1],y_train)

pred=reg.predict(X_valid[features1])

print('multiple linear regression 1')
metrics2.evaluate(reg, X_train[features1], y_train, X_valid[features1], y_valid, pred, price_lambda)

multiple linear regression 1
Mean Squared Error 25,492,063,234.17494
Root Mean Squared Error 159,662.34131
Root Mean Squared Log Error 0.34042
R squared training 0.488
R squared testing 0.42
Adjusted-R squared training 0.488
Adjusted-R squared testing 0.42
intercept [36.36455186]
coefficient [[ 0.88856235  1.42854498 -7.21184417  1.56361903]]


### Viacnásobná lineárna regresia - part2

In [29]:
features2 = feature_selection2.select_features_SFS(X_train, y_train, linear_model.LinearRegression)
reg=linear_model.LinearRegression()
reg.fit(X_train[features2],y_train)

pred=reg.predict(X_valid[features2])

print('multiple linear regression 2')
metrics2.evaluate(reg, X_train[features2], y_train, X_valid[features2], y_valid, pred, price_lambda)

multiple linear regression 2
Mean Squared Error 15,116,743,215.97967
Root Mean Squared Error 122,950.16558
Root Mean Squared Log Error 0.24937
R squared training 0.736
R squared testing 0.686
Adjusted-R squared training 0.736
Adjusted-R squared testing 0.686
intercept [-414.51232489]
coefficient [[ 0.61122557  0.47263888  0.59441788  0.52604449  1.28146307 -0.02351915
   0.36593246 10.28908517  1.91076229 -5.20095952]]


#### k-násobná krížová validácia pri viacnásobnej lineárnej regresii

In [30]:
reg=linear_model.LinearRegression()
k = 5

X_try = pd.concat([X_train[features2], X_valid[features2]])
y_try = np.concatenate((np.array(y_train).reshape(-1,1), np.array(y_valid).reshape(-1,1)), axis=0)

scores = cross_val_score(reg,  X_try[features2], np.array(y_try).reshape(-1,1), cv=k)
print("Cross-validated scores:", scores.mean())
k = 10
scores = cross_val_score(reg,  X_try[features2], np.array(y_try).reshape(-1,1), cv=k)
print("Cross-validated scores:", scores.mean())

Cross-validated scores: 0.7234398406908262
Cross-validated scores: 0.7234453567378276


### Viacnásobná lineárna regresia - part3

In [31]:
features3 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)
reg=linear_model.LinearRegression()
reg.fit(X_train[features3],y_train)

pred=reg.predict(X_valid[features3])

print('multiple linear regression 3')
metrics2.evaluate(reg, X_train[features3], y_train, X_valid[features3], y_valid, pred, price_lambda)

Optimum number of features: 24 (all features: 27)
Score with 24 features: 0.692713
multiple linear regression 3
Mean Squared Error 14,951,839,279.03722
Root Mean Squared Error 122,277.71375
Root Mean Squared Log Error 0.24667
R squared training 0.745
R squared testing 0.693
Adjusted-R squared training 0.744
Adjusted-R squared testing 0.691
intercept [-391.02540102]
coefficient [[-0.19373155  0.32346229  0.46483595 -1.45749151  0.33550029  2.80408969
   0.49387285  0.51812473  1.24387997  2.39437133  0.01128841 -0.02608081
  -0.17237259 -0.29776993 -0.26852878  0.04304076 -0.22111437 -0.1367936
   0.29852013 10.42974166  0.27572955  1.97476794 -3.29043902  3.21229003]]


### Polynomiálna regresia 2. stupňa 

In [32]:
features1 = feature_selection2.select_features_SFS(X_train, y_train, linear_model.LinearRegression)

polyfeat=PolynomialFeatures(degree=2)
xtrain_poly=polyfeat.fit_transform(X_train[features1])
xvalid_poly=polyfeat.fit_transform(X_valid[features1])

np_y_train = np.array(y_train).reshape(-1,1)
np_y_valid = np.array(y_valid).reshape(-1,1)

poly=linear_model.LinearRegression()
poly.fit(xtrain_poly,np_y_train)
polypred=poly.predict(xvalid_poly)

print('Polynomial regresion - grade 2')

mean_squared_error=metrics.mean_squared_error(inv_boxcox(np_y_valid, price_lambda),inv_boxcox(polypred, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(np_y_valid, price_lambda),inv_boxcox(polypred, price_lambda))

r2_train = poly.score(xtrain_poly, np_y_train)
r2_test = poly.score(xvalid_poly, np_y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train[features1].shape[0] - 1)) / (X_train[features1].shape[0] - X_train[features1].shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid[features1].shape[0] - 1)) / (X_valid[features1].shape[0] - X_valid[features1].shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 5))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),5))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),5))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Polynomial regresion - grade 2
Mean Squared Error 12807817726.0493
Root Mean Squared Error 113171.62951
Root Mean Squared Log Error 0.23376
R squared training 0.776
R squared testing 0.727
Adjusted-R squared training 0.775
Adjusted-R squared testing 0.727


### Polynomiálna regresia 3. stupňa

In [33]:
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)

polyfeat=PolynomialFeatures(degree=3)
xtrain_poly=polyfeat.fit_transform(X_train[features1])
xvalid_poly=polyfeat.fit_transform(X_valid[features1])
np_y_train = np.array(y_train).reshape(-1,1)
np_y_valid = np.array(y_valid).reshape(-1,1)

poly=linear_model.LinearRegression()
poly.fit(xtrain_poly,np_y_train)
polypred=poly.predict(xvalid_poly)

print('Polynomial regresion - grade 3')
mean_squared_error=metrics.mean_squared_error(inv_boxcox(np_y_valid, price_lambda),inv_boxcox(polypred, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(np_y_valid, price_lambda),inv_boxcox(polypred, price_lambda))

r2_train = poly.score(xtrain_poly, np_y_train)
r2_test = poly.score(xvalid_poly, np_y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train[features1].shape[0] - 1)) / (X_train[features1].shape[0] - X_train[features1].shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid[features1].shape[0] - 1)) / (X_valid[features1].shape[0] - X_valid[features1].shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 5))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),5))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),5))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Optimum number of features: 24 (all features: 27)
Score with 24 features: 0.692713
Polynomial regresion - grade 3
Mean Squared Error 607625475846570.1
Root Mean Squared Error 24650060.36193
Root Mean Squared Log Error 0.36879
R squared training 0.812
R squared testing 0.234
Adjusted-R squared training 0.811
Adjusted-R squared testing 0.23


In [34]:
features1

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'zipcode_1',
 'zipcode_2',
 'zipcode_3',
 'zipcode_4',
 'zipcode_5',
 'zipcode_6',
 'zipcode_7',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'price_per_sqft']

### Regresný rozhodovací strom

In [35]:
regr_1 = DecisionTreeRegressor(max_depth=10)
regr_1.fit(X_train, y_train)

y_1 = regr_1.predict(X_valid)
mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_1, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_1, price_lambda))

r2_train = regr_1.score(X_train, y_train)
r2_test = regr_1.score(X_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train.shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid.shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Mean Squared Error 10734684770.04
Root Mean Squared Error 103608.32
Root Mean Squared Log Error 0.22
R squared training 0.894
R squared testing 0.767
Adjusted-R squared training 0.893
Adjusted-R squared testing 0.766


### Regresný rozhodovací strom - časť 2

In [36]:
regr_1 = DecisionTreeRegressor(max_depth=8)
regr_1.fit(X_train, y_train)

y_1 = regr_1.predict(X_valid)
mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_1, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_1, price_lambda))

r2_train = regr_1.score(X_train, y_train)
r2_test = regr_1.score(X_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train.shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid.shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Mean Squared Error 10964277725.19
Root Mean Squared Error 104710.45
Root Mean Squared Log Error 0.21
R squared training 0.845
R squared testing 0.769
Adjusted-R squared training 0.845
Adjusted-R squared testing 0.767


### Regresný rozhodovací strom  + GridSearch

In [10]:
hyperparams = {
    'criterion': ['mse', 'friedman_mse', 'mae'],
    'max_depth': np.arange(5,15,1),
    'max_features': np.arange(8,27,1),
    'max_leaf_nodes': [None],
    'min_impurity_decrease': [0.0],
    'min_impurity_split': [None],
    'min_samples_leaf': [1],
    'min_samples_split': [2],
    'min_weight_fraction_leaf': [0.0],
    'splitter': ['best']
}

trees_obj = tuning3.Trees(X_train, X_valid, y_train, y_valid, price_lambda)
df_results1 = trees_obj.grid_search(DecisionTreeRegressor, hyperparams, 50)
df_results1

Iteration 0 in process...done
Iteration 1 in process...done
Iteration 2 in process...done
Iteration 3 in process...done
Iteration 4 in process...done
Iteration 5 in process...done
Iteration 6 in process...done
Iteration 7 in process...done
Iteration 8 in process...done
Iteration 9 in process...done
Iteration 10 in process...done
Iteration 11 in process...done
Iteration 12 in process...done
Iteration 13 in process...done
Iteration 14 in process...done
Iteration 15 in process...done
Iteration 16 in process...done
Iteration 17 in process...done
Iteration 18 in process...done
Iteration 19 in process...done
Iteration 20 in process...done
Iteration 21 in process...done
Iteration 22 in process...done
Iteration 23 in process...done
Iteration 24 in process...done
Iteration 25 in process...done
Iteration 26 in process...done
Iteration 27 in process...done
Iteration 28 in process...done
Iteration 29 in process...done
Iteration 30 in process...done
Iteration 31 in process...done
Iteration 32 in pr

,index,MSE,RMSE,RMSLE,R2_train,R2_test,Adj_R2_train,Adj_R2_test,params,delta_R2
0,47,11714950313.70000,108235.62000,0.22000,0.80800,0.75300,0.80700,0.75100,"{'criterion': 'mse', 'max_depth': 7, 'max_feat...",0.05500
1,46,11731244630.13000,108310.87000,0.22000,0.80700,0.75000,0.80700,0.74900,"{'criterion': 'mse', 'max_depth': 7, 'max_feat...",0.05700
2,49,11866068088.75000,108931.48000,0.22000,0.80700,0.75100,0.80700,0.75000,"{'criterion': 'mse', 'max_depth': 7, 'max_feat...",0.05600
3,37,12114270103.93000,110064.84000,0.22000,0.78900,0.74700,0.78900,0.74500,"{'criterion': 'mse', 'max_depth': 6, 'max_feat...",0.04200
4,45,12138813054.46000,110176.28000,0.22000,0.80400,0.74800,0.80300,0.74600,"{'criterion': 'mse', 'max_depth': 7, 'max_feat...",0.05600
5,34,12239454025.57000,110632.07000,0.23000,0.78200,0.74500,0.78200,0.74300,"{'criterion': 'mse', 'max_depth': 6, 'max_feat...",0.03700
6,35,12266824640.61000,110755.70000,0.23000,0.78800,0.74400,0.78800,0.74200,"{'criterion': 'mse', 'max_depth': 6, 'max_feat...",0.04400
7,48,12346201449.82000,111113.46000,0.23000,0.80600,0.74500,0.80500,0.74300,"{'criterion': 'mse', 'max_depth': 7, 'max_feat...",0.06100
8,36,12384899743.48000,111287.46000,0.23000,0.78800,0.74100,0.78800,0.73900,"{'criterion': 'mse', 'max_depth': 6, 'max_feat...",0.04700
9,33,12719005042.10000,112778.57000,0.23000,0.78600,0.73800,0.78500,0.73700,"{'criterion': 'mse', 'max_depth': 6, 'max_feat...",0.04800


### Náhodný les

In [19]:
reg_rf = RandomForestRegressor(n_estimators=100, max_depth=8)
reg_rf.fit(X_train, y_train)

y_hat = reg_rf.predict(X_valid)
mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_hat, price_lambda))

r2_train = reg_rf.score(X_train, y_train)
r2_test = reg_rf.score(X_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train.shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid.shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Mean Squared Error 19878363316.99
Root Mean Squared Error 140990.65
Root Mean Squared Log Error 0.2
R squared training 0.892
R squared testing 0.841
Adjusted-R squared training 0.892
Adjusted-R squared testing 0.84


### Náhodný les + RandomSearch

In [11]:
hyperparams = {
#     'bootstrap': True,
#     'criterion': 'mse',
    'max_depth': np.arange(8,20,1),
    'max_features': np.arange(8,27,1),
#     'max_leaf_nodes': None,
#     'min_impurity_decrease': 0.0,
#     'min_impurity_split': None,
#     'min_samples_leaf': 1,
#     'min_samples_split': 2,
#     'min_weight_fraction_leaf': 0.0,
    'n_estimators': [10,50,200,700,1000],
#     'n_jobs': None,
#     'oob_score': False,
#     'random_state': None,
#     'verbose': 0,
#     'warm_start': False
}

df_results2 = trees_obj.random_search(RandomForestRegressor, hyperparams, 50)
df_results2

Iteration 0 in process...done
Iteration 1 in process...done
Iteration 2 in process...done
Iteration 3 in process...done
Iteration 4 in process...done
Iteration 5 in process...done
Iteration 6 in process...done
Iteration 7 in process...done
Iteration 8 in process...done
Iteration 9 in process...done
Iteration 10 in process...done
Iteration 11 in process...done
Iteration 12 in process...done
Iteration 13 in process...done
Iteration 14 in process...done
Iteration 15 in process...done
Iteration 16 in process...done
Iteration 17 in process...done
Iteration 18 in process...done
Iteration 19 in process...done
Iteration 20 in process...done
Iteration 21 in process...done
Iteration 22 in process...done
Iteration 23 in process...done
Iteration 24 in process...done
Iteration 25 in process...done
Iteration 26 in process...done
Iteration 27 in process...done
Iteration 28 in process...done
Iteration 29 in process...done
Iteration 30 in process...done
Iteration 31 in process...done
Iteration 32 in pr

,index,MSE,RMSE,RMSLE,R2_train,R2_test,Adj_R2_train,Adj_R2_test,params,delta_R2
0,27,7621619201.47000,87301.89000,0.18000,0.97700,0.83400,0.97700,0.83300,"{'max_depth': 17, 'max_features': 20, 'n_estim...",0.14300
1,26,7629004202.85000,87344.17000,0.18000,0.98100,0.83300,0.98100,0.83200,"{'max_depth': 19, 'max_features': 21, 'n_estim...",0.14800
2,13,7630536994.30000,87352.94000,0.18000,0.96500,0.83500,0.96400,0.83400,"{'max_depth': 14, 'max_features': 22, 'n_estim...",0.13000
3,31,7636327240.98000,87386.08000,0.18000,0.96500,0.83400,0.96500,0.83300,"{'max_depth': 14, 'max_features': 25, 'n_estim...",0.13100
4,46,7637201580.03000,87391.08000,0.18000,0.98100,0.83300,0.98100,0.83200,"{'max_depth': 19, 'max_features': 20, 'n_estim...",0.14800
5,49,7641788759.22000,87417.33000,0.18000,0.97800,0.83300,0.97800,0.83100,"{'max_depth': 17, 'max_features': 25, 'n_estim...",0.14500
6,25,7651677443.35000,87473.87000,0.18000,0.97000,0.83400,0.97000,0.83300,"{'max_depth': 15, 'max_features': 22, 'n_estim...",0.13600
7,23,7655941535.04000,87498.24000,0.18000,0.96400,0.83400,0.96400,0.83300,"{'max_depth': 14, 'max_features': 22, 'n_estim...",0.13000
8,40,7656723856.83000,87502.71000,0.18000,0.97700,0.83200,0.97700,0.83100,"{'max_depth': 17, 'max_features': 26, 'n_estim...",0.14500
9,12,7690458873.56000,87695.26000,0.18000,0.95700,0.83500,0.95600,0.83400,"{'max_depth': 13, 'max_features': 22, 'n_estim...",0.12200


In [18]:
for i in range(10):
    print(df_results2['params'][i])

{'max_depth': 17, 'max_features': 20, 'n_estimators': 1000}
{'max_depth': 19, 'max_features': 21, 'n_estimators': 1000}
{'max_depth': 14, 'max_features': 22, 'n_estimators': 1000}
{'max_depth': 14, 'max_features': 25, 'n_estimators': 1000}
{'max_depth': 19, 'max_features': 20, 'n_estimators': 700}
{'max_depth': 17, 'max_features': 25, 'n_estimators': 700}
{'max_depth': 15, 'max_features': 22, 'n_estimators': 200}
{'max_depth': 14, 'max_features': 22, 'n_estimators': 700}
{'max_depth': 17, 'max_features': 26, 'n_estimators': 200}
{'max_depth': 13, 'max_features': 22, 'n_estimators': 700}


### Náhodný les + GridSearch

In [ ]:
hyperparams = {
#     'bootstrap': True,
#     'criterion': 'mse',
    'max_depth': np.arange(13,19,1),
    'max_features': np.arange(20,27,1),
#     'max_leaf_nodes': None,
#     'min_impurity_decrease': 0.0,
#     'min_impurity_split': None,
#     'min_samples_leaf': 1,
#     'min_samples_split': 2,
#     'min_weight_fraction_leaf': 0.0,
    'n_estimators': [700, 800,1000],
#     'n_jobs': None,
#     'oob_score': False,
#     'random_state': None,
#     'verbose': 0,
#     'warm_start': False
}

df_results2 = trees_obj.grid_search(RandomForestRegressor, hyperparams, 50)
df_results2

### LinearSVR

In [21]:
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)

clf = LinearSVR(tol=1e-4, loss='squared_epsilon_insensitive')
X_my_train = X_train[features1]#.values.reshape(-1,1)
X_my_valid = X_valid[features1]#.values.reshape(-1,1)

clf.fit(X_my_train, y_train)

y_hat = clf.predict(X_my_valid)
mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_hat, price_lambda))

r2_train = clf.score(X_my_train, y_train)
r2_test = clf.score(X_my_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_my_train.shape[0] - 1)) / (X_my_train.shape[0] - X_my_train.shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_my_valid.shape[0] - 1)) / (X_my_valid.shape[0] - X_my_valid.shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Squared Error 46836652678.26
Root Mean Squared Error 216417.77
Root Mean Squared Log Error 0.32
R squared training 0.659
R squared testing 0.6
Adjusted-R squared training 0.659
Adjusted-R squared testing 0.598


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### RandomSearch pre LinearSVR

In [7]:
param_grid = {
#      "C": np.random.uniform(1.2, 2.0, 100),
#     "dual":[True],
#     "epsilon":np.random.uniform(0.1, 0.2, 100),
#     "fit_intercept":[True],
#     "intercept_scaling":[1],
    "loss":['epsilon_insensitive', 'squared_epsilon_insensitive'],
    "max_iter":[10000],
    "tol":np.random.uniform(1e-6, 5e-4, 100),
#     "verbose":[0]
}

svr_obj = tuning3.LinearSVR(X_train, X_valid, y_train, y_valid, price_lambda)
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)
df_results = svr_obj.random_search(LinearSVR, param_grid, 50, features1)
df_results

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,index,MSE,RMSE,RMSLE,R2_train,R2_test,Adj_R2_train,Adj_R2_test,params
0,0,38524914674.86000,196277.65000,0.27000,0.72700,0.70600,0.72700,0.70500,"{'loss': 'squared_epsilon_insensitive', 'max_i..."


### SGD + Nystroem

In [22]:
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)

clf = SGDRegressor(tol=1e-3, penalty='l1', shuffle=False)
feature_map_nystroem = Nystroem(gamma=.2, random_state=1, n_components=300)

data_transformed_train = feature_map_nystroem.fit_transform(X_train[features1])
data_transformed_valid = feature_map_nystroem.transform(X_valid[features1])

clf.fit(data_transformed_train, np.ravel(y_train))
y_hat = clf.predict(data_transformed_valid)


mean_squared_error=metrics.mean_squared_error(inv_boxcox(y_valid, price_lambda),inv_boxcox(y_hat, price_lambda))
rmlse = metrics.mean_squared_log_error(inv_boxcox(y_valid, price_lambda), inv_boxcox(y_hat, price_lambda))

r2_train = clf.score(data_transformed_train, y_train)
r2_test = clf.score(data_transformed_valid, y_valid)
adj_r2_train = 1 - (((1 - r2_train) * (X_train.shape[0] - 1)) / (X_train.shape[0] - X_train[features1].shape[1] - 1))
adj_r2_test = 1 - (((1 - r2_test) * (X_valid.shape[0] - 1)) / (X_valid.shape[0] - X_valid[features1].shape[1] - 1))

print('Mean Squared Error', round(mean_squared_error, 2))
print('Root Mean Squared Error', round(np.sqrt(mean_squared_error),2))
print('Root Mean Squared Log Error', round(np.sqrt(rmlse),2))
print('R squared training',round(r2_train,3))
print('R squared testing',round(r2_test,3))
print('Adjusted-R squared training', round(adj_r2_train,3))
print('Adjusted-R squared testing', round(adj_r2_test,3))

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436
Mean Squared Error 120493650338.01
Root Mean Squared Error 347121.95
Root Mean Squared Log Error 0.56
R squared training -0.221
R squared testing -0.192
Adjusted-R squared training -0.223
Adjusted-R squared testing -0.198


### SGD + RandomSearch

In [6]:
param_grid = {
    'alpha': np.random.uniform(1e-5,1e-3,100),
#     'average': [True,False],
    'early_stopping': [True],
#     'epsilon': [0.1],
#     'eta0': np.random.uniform(0.001,0.1,100),
#     'fit_intercept': [True],
    'l1_ratio': np.random.uniform(0.1,0.9,100),
    'learning_rate': ['constant', 'adaptive','invscaling'],
    'loss': ['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
    'max_iter': [10000],
    'n_iter_no_change': [5],
    'penalty': ['l1','l2','elasticnet'],
    'power_t': np.random.uniform(0.15,0.4,100),
    'shuffle': [False],
    "tol":np.random.uniform(1e-4, 1e-2, 100),
    'validation_fraction': np.arange(0.1, 0.3, 0.05),
}
sgd_obj = tuning3.SGD(X_train, X_valid, y_train, y_valid, price_lambda)
features1 = feature_selection2.select_features_RFE(X_train, X_valid, y_train, y_valid)
df_results2 = sgd_obj.random_search(SGDRegressor, param_grid, 50, features1)
df_results2

Optimum number of features: 22 (all features: 27)
Score with 22 features: 0.754436


,index,MSE,RMSE,RMSLE,R2_train,R2_test,Adj_R2_train,Adj_R2_test,params
0,0,106324648296.28000,326074.61000,0.51000,-0.00200,-0.00100,-0.02200,-0.07500,"{'alpha': 0.00045859496912911034, 'early_stopp..."


In [14]:
df_results2["params"][3]

{'alpha': 0.00010649479903447461,
 'early_stopping': True,
 'l1_ratio': 0.2730629352844155,
 'learning_rate': 'constant',
 'loss': 'huber',
 'max_iter': 10000,
 'n_iter_no_change': 5,
 'penalty': 'l2',
 'power_t': 0.1764792717720897,
 'shuffle': False,
 'tol': 0.008364349005682366,
 'validation_fraction': 0.20000000000000004}